In [170]:
from tools import analysistools as atools
import nanoparticle
from nanoparticle import Ligand, NanoParticle
import math
from membranesimulation import MembraneSimulation
import numpy as np
from tools import icosatiler
from tools import vectools
import k3d

In [166]:
def crossProd(u,v):
    return ((u[1]*v[2]-u[2]*v[1]),(u[2]*v[0]-u[0]*v[2]),(u[0]*v[1]-u[1]*v[0]))

In [2]:
def makeParticleModelFromPhenome(particle):
    points = []
    plot = k3d.plot()
    i = 1
    maxEps = 0
    for l in particle.ligands:
        if l.eps>maxEps:
            maxEps = l.eps
    for l in particle.ligands:
        if l.eps > 0.0:
            lx = l.rad*np.sin(l.polAng)*np.cos(l.aziAng)
            ly = l.rad*np.sin(l.polAng)*np.sin(l.aziAng)
            lz = l.rad*np.cos(l.polAng)
            lc = '0x' + ('#%02x%02x%02x' % (0, int((l.eps/maxEps)*255.0), 0))[1:]
            
            plot += k3d.points([lx,ly,lz],point_size=1,color=int(lc, 16))
            #plot += k3d.points([lx,ly,lz],point_size=1,color=int(0x00ffff*(l.eps/maxEps)))
            plot += k3d.text(str(i),[lx,ly,lz],color=0xffffff)
            i+=1
    plot += k3d.points([0,0,0],point_size=7,color=0xff0000)
    return plot

In [3]:
def buildCoveredParticle(eps,rad):
    particle = NanoParticle()
    lPos = icosatiler.cover72SpherePolar(rad)
    for v in lPos:
        particle.addLigand(Ligand(eps,1,v[0],v[1],v[2]))
    return particle

In [4]:
def spaceIsOccupied(particle,v):
    for l in particle.ligands:
        d = atools.greatArcDist((l.polAng,l.aziAng),(v[1],v[2]),particle.sig)
        if abs(d) < l.size:
            return True
    return False

In [106]:
def getClosestIcos(particle,v):
    lPos = icosatiler.cover72SpherePolar(particle.sig)
    closest = v
    minD = 1e8
    for l in lPos:
        v1 = icosatiler.sphPol2Crt(l) 
        v2 = icosatiler.sphPol2Crt(v)
        d = np.sum([j*j for j in np.subtract(v1,v2)])
        if abs(d) < minD:
            minD = d
            closest = l
    return closest

In [146]:
# lPos = icosatiler.cover72SpherePolar(4)
# dists = []
# minD = 1e8
# for v in lPos:
#     for l in lPos:
#         if l != v:
#             v1 = icosatiler.sphPol2Crt(l) 
#             v2 = icosatiler.sphPol2Crt(v)
#             d = np.sum([j*j for j in np.subtract(v1,v2)])
#             if abs(d) < minD:
#                 minD = d
#     dists.append(np.sqrt(minD))
# print np.mean(dists)

1.5990747678128467


In [52]:
def addLigand(particle,eps,v,lock=False):
    if lock:
        v = getClosestIcos(particle,v)
            
    if not spaceIsOccupied(particle,v):
        particle.addLigand(Ligand(eps,1,v[0],v[1],v[2]))
    return particle

In [53]:
def addBelt(particle,eps,rad,n,phi,lock=False):
    step = np.pi*2/float(n)
    for i in range(n):
        v = (rad,step*i,phi)
        particle = addLigand(particle,eps,v,lock)
    return particle

In [54]:
def getArcAngle(dist,rad):
    return dist/rad

In [380]:
def addRing(particle,eps,rad,pos,n,spacing,lock=False):
    step = np.pi*2/float(n)
    if lock:
        origin = getClosestIcos(particle,(rad,pos[0],pos[1]))
        pos = (origin[1],origin[2])
    for i in range(n):
        tD = getArcAngle(spacing,rad)
        tP = step*i
        v = (rad,tD,tP)
        vC = icosatiler.sphPol2Crt(v)
        v1 = icosatiler.sphPol2Crt((rad,0,0))
        v2 = icosatiler.sphPol2Crt((rad,pos[0],pos[1]))
        v3 = crossProd(v1,v2)
        if np.sum([j*j for j in v3])== 0.0:
            #it's parallel!
            particle = addLigand(particle,eps,v,lock)
            continue
        dp = np.dot(v1,v2)
        print dp
        ang = np.arccos((dp)/(np.sqrt(np.sum([j*j for j in v1]))*np.sqrt(np.sum([j*j for j in v2]))))
        print ang
        rM = vectools.buildERMatrix(v3, ang)
        vC = np.dot(rM,vC)
        v = icosatiler.crt2SphPol(vC)
        particle = addLigand(particle,eps,v,lock)
        print v
    return particle

In [381]:
def addPatch(particle,eps,rad,pos,n,spacing,lock=False):
    if lock:
        origin = getClosestIcos(particle,(rad,pos[0],pos[1]))
        pos = (origin[1],origin[2])
    particle = addLigand(particle,eps,(rad,pos[0],pos[1]),lock)
    particle = addRing(particle,eps,rad,pos,n,spacing)
    return particle

In [382]:
p = NanoParticle()
m = 3
step = np.pi*2/float(m)
for i in range(m):
    p = addPatch(p,6,4,(i*step,np.pi/2.0),5,1.6,lock=False)
#p = addPatch(p,6,4,(np.pi/2.0,np.pi/2.0),5,1.6,lock=True)
# p = addRing(p,10,4,(lPos[0][1],lPos[0][2]),6,0,lock=False)

-7.9999999999999964
2.0943951023931953
(4.0, 2.0493890789306235, 1.1166335589906586)
-7.9999999999999964
2.0943951023931953
(4.0, 2.467495263213243, 1.376793531201165)
-7.9999999999999964
2.0943951023931953
(4.0, 2.289964173754803, 2.0028614105814286)
-7.9999999999999964
2.0943951023931953
(4.0, 1.836203460053994, 1.9033706869414873)
-7.9999999999999964
2.0943951023931953
(4.0, 1.7110459817649413, 1.4489649875339103)
-8.000000000000007
2.094395102393196
(4.0, 2.0493890789306244, -1.1166335589906584)
-8.000000000000007
2.094395102393196
(4.000000000000001, 1.7110459817649422, -1.4489649875339106)
-8.000000000000007
2.094395102393196
(4.0, 1.8362034600539947, -1.9033706869414875)
-8.000000000000007
2.094395102393196
(4.0, 2.2899641737548038, -2.002861410581429)
-8.000000000000007
2.094395102393196
(4.000000000000001, 2.467495263213244, -1.376793531201165)


In [383]:
makeParticleModelFromPhenome(p)

Plot(antialias=True, background_color=16777215, grid=[-1, -1, -1, 1, 1, 1], height=512, object_ids=[120962484752, 120961713872, 120962484560, 120962662224, 120961714640, 120962485200, 120962661200, 120962781840, 120962782032, 120962783888, 120962785168, 120962784656, 120962662032, 120962784528, 120962781456, 120962781264, 120962781968, 120962783056, 120962782096, 120962783440, 120962781520, 120962783632, 120962782288, 120962785040, 120962782928, 120962784400, 120962782544, 120962782992, 120962860048, 120962782864, 120962860112, 120962859536, 120962781584, 120962784144, 120962860496, 120962859792, 120962483600])